Experimenting with DTW and plotting DTW mapping

In [ ]:
import django_initialiser
import time
import math
from api.models import Song
import numpy as np
from fastdtw import fastdtw
import matplotlib.pyplot as plt

In [ ]:
def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))

    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            dtw_matrix[i, j] = cost + np.min([dtw_matrix[i-1, j],       # deletion
                                              dtw_matrix[i, j-1],       # insertion
                                              dtw_matrix[i-1, j-1]])    # match
    return dtw_matrix[n, m]

In [ ]:
def dtw_mat(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))

    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            dtw_matrix[i, j] = cost + np.min([dtw_matrix[i-1, j],       # deletion
                                              dtw_matrix[i, j-1],       # insertion
                                              dtw_matrix[i-1, j-1]])    # match
    return dtw_matrix

In [ ]:
# Break songs into segments of same length as query
def segmented_lookup(query):
    t = time.process_time()
    # Retrieve all songs from db
    songs = Song.objects.all()

    results = []
    query_len = len(query)

    for song in songs:
        song_notes = list(map(int, song.note_sequence.split(',')))
        song_len = len(song_notes)

        min_distance = np.inf
        min_first_index = 0
        min_last_index = 0
        min_path = []

        # Get segments
        for i in range(math.ceil(song_len / query_len)):
            first_note_index = i * query_len
            last_note_index = (i + 1) * query_len

            segment = song_notes[first_note_index:last_note_index]

            # Last note index can be out of range but array slicing is safe against that
            distance, path = fastdtw(segment, query)
            if distance < min_distance:
                min_distance = distance
                min_first_index = first_note_index
                min_last_index = last_note_index
                min_path = path

        results.append((min_distance, song, [min_first_index, min_last_index], min_path))

    # Sort in descending order by the longest subsequence length
    results.sort(key=lambda tup: tup[0])
    t = time.process_time() - t
    print(f'Done in {t} seconds')
    return results


In [ ]:
elise = Song.objects.filter(
    title__contains='elise'
)
input_search = [76, 75, 76, 75, 76, 71, 74, 72, 69, 57]
input_search2 = [74, 73, 74, 73, 74, 69, 72, 70, 67, 55]
elise_notes = list(map(int, elise[0].note_sequence.split(',')))

In [ ]:
res = segmented_lookup(input_search2)
for i in range(0, 3):
    print(res[i][0], res[i][1].title, f'at [{res[i][2][0]}, {res[i][2][1]}]')

In [ ]:
# Get second result's song notes again
inspected_res = 1
song_notes = list(map(int, res[inspected_res][1].note_sequence.split(',')))
segment = song_notes[res[inspected_res][2][0]:res[inspected_res][2][1]]

print(res[inspected_res][3])

for mapping in res[inspected_res][3]:
    x1 = mapping[1]
    y1 = segment[mapping[1]]
    x2 = mapping[0]
    y2 = input_search2[mapping[0]]
    color = 'limegreen'

    # print([x1, y1], [x2, y2])
    plt.plot([x1, x2], [y1, y2], color=color, linestyle='-', linewidth=1)
plt.plot(input_search2, label='Search', linewidth=2)
plt.plot(segment, label='Song', linewidth=2)
plt.locator_params(axis="both", integer=True, tight=True)
plt.legend()

plt.savefig('foo.png')
plt.show()

In [ ]:
# Trying to save graph to db
from pathlib import Path
from django.core.files import File
from api.models import DTWResultGraph
path = Path('./foo.png')
dtw_graph = DTWResultGraph(result_index=0)
with path.open(mode='rb') as f:
    dtw_graph.graph = File(f, name=path.name)
    dtw_graph.save()

In [ ]:
DTWResultGraph.objects.all().delete()